<a href="https://colab.research.google.com/github/Shukals/CSecuirity/blob/main/CS02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyshark


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.5 MB/s eta 0:00:00


In [2]:
pip install nest_asyncio


In [4]:
!apt update && apt install -y tshark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.9 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,303 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,631 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,742 kB]
Hit:13 https://ppa.launchpadcontent

In [5]:
!DEBIAN_FRONTEND=noninteractive apt install -y tshark


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tshark is already the newest version (3.6.2-2).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [6]:
!tshark --version

Running as user "root" and group "root". This could be dangerous.
TShark (Wireshark) 3.6.2 (Git v3.6.2 packaged as 3.6.2-2)

Copyright 1998-2022 Gerald Combs <gerald@wireshark.org> and contributors.
License GPLv2+: GNU GPL version 2 or later <https://www.gnu.org/licenses/gpl-2.0.html>
This is free software; see the source for copying conditions. There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Compiled (64-bit) using GCC 11.2.0, with libpcap, with POSIX capabilities
(Linux), with libnl 3, with GLib 2.71.2, with zlib 1.2.11, with Lua 5.2.4, with
GnuTLS 3.7.3 and PKCS #11 support, with Gcrypt 1.9.4, with MIT Kerberos, with
MaxMind DB resolver, with nghttp2 1.43.0, with brotli, with LZ4, with Zstandard,
with Snappy, with libxml2 2.9.12, with libsmi 0.4.8.

Running on Linux 6.1.85+, with Intel(R) Xeon(R) CPU @ 2.20GHz (with SSE4.2),
with 12978 MB of physical memory, with GLib 2.72.4, with zlib 1.2.11, with
libpcap 1.10.1 (with TPACKET_V3), with c-are

In [18]:
import pyshark
from collections import defaultdict

def analyze_tcp_flows(pcap_file):
    tcp_flows = defaultdict(lambda: {'packet_count': 0, 'total_bytes': 0})

    # Read the pcap file
    cap = pyshark.FileCapture(pcap_file, display_filter='tcp')

    for packet in cap:
        try:
            src_ip = packet.ip.src
            dst_ip = packet.ip.dst
            src_port = packet.tcp.srcport
            dst_port = packet.tcp.dstport
            flow_key = (src_ip, src_port, dst_ip, dst_port)

            tcp_flows[flow_key]['packet_count'] += 1
            tcp_flows[flow_key]['total_bytes'] += int(packet.length)
        except AttributeError:
            continue  # Skip packets that don't have the expected fields

    cap.close()

    # Print the summary
    print("TCP Flow Summary:")
    print("{:<20} {:<8} {:<20} {:<8} {:<12} {:<12}".format("Source IP", "Src Port", "Destination IP", "Dst Port", "Packet Count", "Total Bytes"))
    print("-" * 80)
    for (src_ip, src_port, dst_ip, dst_port), stats in tcp_flows.items():
        print(f"{src_ip:<20} {src_port:<8} {dst_ip:<20} {dst_port:<8} {stats['packet_count']:<12} {stats['total_bytes']:<12}")

if __name__ == "__main__":
    pcap_file = '/content/lab2.pcap'  # Change this to your pcap file path
    analyze_tcp_flows(pcap_file)

TCP Flow Summary:
Source IP            Src Port Destination IP       Dst Port Packet Count Total Bytes 
--------------------------------------------------------------------------------
192.168.32.232       50511    8.8.8.8              443      1            55          
8.8.8.8              443      192.168.32.232       50511    1            54          
192.168.32.232       50569    34.144.254.29        443      3            163         
34.144.254.29        443      192.168.32.232       50569    2            120         
192.168.32.232       50385    34.144.254.29        443      2            110         
34.144.254.29        443      192.168.32.232       50385    2            132         
192.168.32.232       50577    52.191.219.104       443      13           8004        
52.191.219.104       443      192.168.32.232       50577    10           4793        
192.168.32.232       50355    34.144.254.29        443      1            55          
34.144.254.29        443      192.168.32.

In [19]:
import pyshark
import requests
from collections import defaultdict

def get_geolocation(ip, api_key):
    url = f"https://ipinfo.io/{ip}/json?token={api_key}"
    try:
        response = requests.get(url)
        data = response.json()
        return data.get('country', 'Unknown'), data.get('region', 'Unknown'), data.get('city', 'Unknown')
    except requests.RequestException:
        return 'Unknown', 'Unknown', 'Unknown'

def analyze_ip_geolocation(pcap_file, api_key):
    ip_locations = defaultdict(lambda: {'count': 0, 'location': ('Unknown', 'Unknown', 'Unknown')})

    cap = pyshark.FileCapture(pcap_file)

    for packet in cap:
        try:
            src_ip = packet.ip.src
            if src_ip not in ip_locations:
                ip_locations[src_ip]['location'] = get_geolocation(src_ip, api_key)
            ip_locations[src_ip]['count'] += 1
        except AttributeError:
            continue

    cap.close()

    print("IP Geolocation Summary:")
    print("{:<20} {:<10} {:<15} {:<15} {:<10}".format("IP Address", "Country", "Region", "City", "Count"))
    print("-" * 75)
    for ip, info in ip_locations.items():
        country, region, city = info['location']
        print(f"{ip:<20} {country:<10} {region:<15} {city:<15} {info['count']:<10}")

if __name__ == "__main__":
    API_KEY = 'ipinfo.io' # Replace with your actual API key
    PCAP_FILE = '/content/lab2.pcap'  # Change to your pcap file path
    analyze_ip_geolocation(PCAP_FILE, API_KEY)


IP Geolocation Summary:
IP Address           Country    Region          City            Count     
---------------------------------------------------------------------------
192.168.32.2         Unknown    Unknown         Unknown         5         
192.168.32.232       Unknown    Unknown         Unknown         114       
34.144.254.29        US         Missouri        Kansas City     25        
8.8.8.8              US         California      Mountain View   11        
8.8.4.4              US         California      Mountain View   4         
52.191.219.104       US         District of Columbia Washington      21        
34.106.86.104        US         Utah            Salt Lake City  21        


In [21]:
import pyshark
from collections import defaultdict
from datetime import datetime

def detect_port_scanning(pcap_file, time_window=5):
    scan_activity = defaultdict(list)

    cap = pyshark.FileCapture(pcap_file, display_filter='tcp')

    for packet in cap:
        try:
            timestamp = float(packet.sniff_timestamp)
            src_ip = packet.ip.src
            dst_ip = packet.ip.dst
            dst_port = packet.tcp.dstport

            scan_activity[(src_ip, dst_ip)].append((timestamp, int(dst_port)))
        except AttributeError:
            continue

    cap.close()

    print("Suspicious Port Scanning Activity:")
    print("{:<20} {:<20} {:<20} {:<10} {:<10}".format("Source IP", "Destination IP", "Ports Scanned", "Start Time", "End Time"))
    print("-" * 80)

    for (src_ip, dst_ip), scans in scan_activity.items():
        scans.sort()
        scanned_ports = []
        start_time, end_time = None, None

        for i in range(len(scans)):
            if start_time is None:
                start_time = scans[i][0]
            end_time = scans[i][0]
            scanned_ports.append(scans[i][1])

            if end_time - start_time > time_window:
                break

        if len(scanned_ports) > 5:
            start_time_str = datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S')
            end_time_str = datetime.fromtimestamp(end_time).strftime('%Y-%m-%d %H:%M:%S')
            print(f"{src_ip:<20} {dst_ip:<20} {sorted(scanned_ports)} {start_time_str:<10} {end_time_str:<10}")

if __name__ == "__main__":
    pcap_file = '/content/lab2.pcap'
    detect_port_scanning(pcap_file)


Suspicious Port Scanning Activity:
Source IP            Destination IP       Ports Scanned        Start Time End Time  
--------------------------------------------------------------------------------
192.168.32.232       52.191.219.104       [443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443] 2025-01-29 07:19:04 2025-01-29 07:19:10
52.191.219.104       192.168.32.232       [50577, 50577, 50577, 50577, 50577, 50577, 50577, 50577, 50577, 50577, 50579, 50579, 50579, 50579, 50579, 50579, 50579, 50579, 50579, 50579] 2025-01-29 07:19:05 2025-01-29 07:19:10
192.168.32.232       34.106.86.104        [443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443, 443] 2025-01-29 07:19:22 2025-01-29 07:19:28
34.106.86.104        192.168.32.232       [50581, 50581, 50581, 50581, 50581, 50581, 50581, 50581, 50581, 50581, 50581, 50583, 50583, 50583, 50583, 50583, 50583] 2025-01-29 07:19:23 2025-01-29 07

In [24]:
import re
import requests

def extract_ip(email_header):
    """Extract the originating IP address from an email header."""
    ip_pattern = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')

    for line in email_header.split("\n"):
        if "Received: from" in line:
            ip_matches = ip_pattern.findall(line)
            for ip in ip_matches:
                if not ip.startswith("10.") and not ip.startswith("192.168.") and not ip.startswith("172."):
                    return ip  # Return the first non-private IP found
    return None

def get_ip_location(ip):
    """Retrieve geolocation information for a given IP address."""
    try:
        response = requests.get(f"https://ipinfo.io/{ip}/json")
        data = response.json()
        return {
            "IP": ip,
            "City": data.get("city", "Unknown"),
            "Region": data.get("region", "Unknown"),
            "Country": data.get("country", "Unknown"),
            "Location": data.get("loc", "Unknown"),
            "ISP": data.get("org", "Unknown")
        }
    except requests.RequestException:
        return {"Error": "Unable to retrieve geolocation data"}

def main():
    email_header = """
    Received: from mail.example.com (mail.example.com. [209.85.220.69])
    by mx.google.com with ESMTPS id x7si12345678qka.123.2024.01.29.12.34.56
    """

    ip = extract_ip(email_header)

    if ip:
        location = get_ip_location(ip)
        print("Originating IP Address:", ip)
        print("Location Information:", location)
    else:
        print("No valid IP address found in the email header.")

if __name__ == "__main__":
    main()

Originating IP Address: 209.85.220.69
Location Information: {'IP': '209.85.220.69', 'City': 'Atlanta', 'Region': 'Georgia', 'Country': 'US', 'Location': '33.7490,-84.3880', 'ISP': 'AS15169 Google LLC'}
